In [8]:
%matplotlib inline
import pandas as pd

#Importo Decisions.csv
decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il primo round
decisions1 = decisions[decisions["Round"] == 9]

#Ho effettuato un controllo esterno per assicurarmi che tutte le tuple vengano prese in considerazione

#Estraggo "ID", "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions1 = decisions1.iloc[:, [0, 1, 3, 4]]

#Da "Decisions.csv" seleziono solo le tuple riguardanti il secondo round
decisions2 = decisions[decisions["Round"] == 10]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]

#Importo TasksQualities per le task assegnate a ciascun WA
queue = pd.read_csv("../Dataset 2.0/TasksQualities.csv", sep = ";")

queue = queue.sort_values(by = "ID")

#Creo un DataFrame vuoto nel quale concatenerò il risultato di ciascuna iterazione che sto per eseguire
totale = pd.DataFrame()
print("end")

fine


In [11]:
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

for i in range(20):
    
    #Faccio coincidere in entrambe le tabelle gli wa scelti (in due round consecutivi)
    tmp1 = decisions1[decisions1["Worker Agent ID"] == i+1]
    tmp2 = decisions2[decisions2["Worker Agent ID"] == i+1]
    tmp1 = pd.merge(tmp1, tmp2, left_on = "Session ID", right_on = "Session ID")
    tmp1 = tmp1.rename(columns = {"Worker Agent ID_x": "Worker Agent ID", "Worker Agent Reputation_x": "Worker Agent Starting Reputation", "Worker Agent Reputation_y": "Worker Agent Observed Reputation"})
    tmp1 = pd.merge(queue, tmp1, left_on = "ID", right_on = "ID")
    tmp1 = tmp1.drop("Worker Agent ID_y", 1)
    tmp1 = tmp1.drop("Session ID", 1)
    tmp1 = tmp1.sort_values(by = "ID")
    
    xtmp = tmp1.iloc[:, 0:93]
    ytmp = tmp1.iloc[:, 93]
    
    reg = LinearRegression().fit(xtmp, ytmp)
    print(f"Worker Agent {i+1}, score: {reg.score(xtmp, ytmp)}\n")
    
    totale = pd.concat([totale, tmp1])
    
    #Training del modello per ogni wa riguardo al round scelto

Worker Agent 1, score: 0.910865278239813

Worker Agent 2, score: 0.9242838574213197

Worker Agent 3, score: 0.9067901846863252

Worker Agent 4, score: 0.8971914980613082

Worker Agent 5, score: 0.8862170041828635

Worker Agent 6, score: 0.9179138001706617

Worker Agent 7, score: 0.9150264119334063

Worker Agent 8, score: 0.9347754003998848

Worker Agent 9, score: 0.9376361679486526

Worker Agent 10, score: 0.94772346474345

Worker Agent 11, score: 0.8950181686861094

Worker Agent 12, score: 0.8966050619240913

Worker Agent 13, score: 0.86660240165

Worker Agent 14, score: 0.8729066574243519

Worker Agent 15, score: 0.8921081547514378

Worker Agent 16, score: 0.9112040475438025

Worker Agent 17, score: 0.9243552187728464

Worker Agent 18, score: 0.9260596783830306

Worker Agent 19, score: 0.9310552887911175

Worker Agent 20, score: 0.9298179458365676



In [13]:
totale = totale.sort_values(by = "ID")

#Preparazione del dataset per il training
x = totale.iloc[:, 0:93]
y = totale.iloc[:, 93]
print("fine")

fine


In [14]:
#Training del modello utilizzando la Regressione Lineare

reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.9537148596502453

In [22]:
#Training del modello utilizzando SVR

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x, y)
regr.score(x, y)

0.788342762977449